In [1]:
import os, sys

sys.path.append(os.path.abspath(os.path.join("../src/")))
from models.Encoder import myself_net

from torchinfo import summary

import torch

in_height = 224
in_width = 304
input_rgb = torch.randn(1, 3, in_height, in_width).cuda()
input_dep = torch.randn(1, 1, in_height, in_width).cuda()

net_mini = myself_net(channels_list=[8, 16, 32, 64, 128]).cuda()
net_mini_dense_depth = net_mini(input_rgb, input_dep)
net_mini_summary = summary(
    model=net_mini,
    input_size=[input_rgb.shape, input_dep.shape],
    col_names=("input_size", "output_size", "num_params", "kernel_size", "mult_adds"),
    depth=2,
)
print("net_mini total params: %0.6f M" % (net_mini_summary.total_params / 1000000.0))
# print(net_mini_summary)

net_tiny = myself_net(channels_list=[16, 32, 64, 128, 256])
net_tiny_summary = summary(
    net_tiny,
    input_size=[input_rgb.shape, input_dep.shape],
    col_names=("input_size", "output_size", "num_params", "kernel_size", "mult_adds"),
    depth=2,
)
print("net_tiny total params: %0.6f M" % (net_tiny_summary.total_params / 1000000.0))
# print(net_tiny_summary)

net_small = myself_net(channels_list=[32, 64, 128, 256, 512])
net_small_summary = summary(
    net_small,
    input_size=[input_rgb.shape, input_dep.shape],
    col_names=("input_size", "output_size", "num_params", "kernel_size", "mult_adds"),
    depth=2,
)
print("net_small total params: %0.6f M" % (net_small_summary.total_params / 1000000.0))
# print(net_small_summary)

net_base = myself_net(channels_list=[64, 128, 256, 512, 1024])
net_base_summary = summary(
    net_base,
    input_size=[input_rgb.shape, input_dep.shape],
    col_names=("input_size", "output_size", "num_params", "kernel_size", "mult_adds"),
    depth=1,
)
print("net_base total params: %0.6f M" % (net_base_summary.total_params / 1000000.0))
print(net_base_summary)

net_mini total params: 0.782658 M
net_tiny total params: 3.004842 M
net_small total params: 11.768442 M
net_base total params: 46.572570 M
Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
myself_net                               [1, 3, 224, 304]          [1, 1, 224, 304]          --                        --                        --
├─Sequential: 1-1                        [1, 3, 224, 304]          [1, 32, 224, 304]         928                       --                        58,835,008
├─Block: 1-2                             [1, 32, 224, 304]         [1, 64, 224, 304]         57,792                    --                        3,909,255,552
├─Block: 1-3                             [1, 64, 224, 304]         [1, 128, 112, 152]        230,272                   --                        3,907,076,864
├─Block: 1-4                             [1, 128, 112, 152]        [1, 256, 56, 76] 

In [2]:
import os, sys

sys.path.append(os.path.abspath(os.path.join("../src/")))
from models.Encoder import up_DKN

from torchinfo import summary

import torch

net = up_DKN(16).cuda()
feature = torch.randn(1, 16, 228, 304).cuda()
depth = torch.randn(1, 1, 228, 304).cuda()
output= net(depth, feature).cuda()

summary(
    net,
    input_size=[(1, 1, 228, 304), (1, 16, 228, 304)],
    col_names=("input_size", "output_size", "num_params", "kernel_size", "mult_adds"),
)

Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Kernel Shape              Mult-Adds
up_DKN                                   [1, 1, 228, 304]          [1, 1, 228, 304]          --                        3                         --
├─Sequential: 1-1                        [1, 1, 228, 304]          [1, 16, 228, 304]         --                        --                        --
│    └─Conv2d: 2-1                       [1, 1, 228, 304]          [1, 16, 228, 304]         16                        [1, 1]                    1,108,992
│    └─BatchNorm2d: 2-2                  [1, 16, 228, 304]         [1, 16, 228, 304]         32                        --                        32
│    └─LeakyReLU: 2-3                    [1, 16, 228, 304]         [1, 16, 228, 304]         --                        --                        --
├─Sequential: 1-2                        [1, 32, 228, 304]         [1, 9, 228, 304]          --   